In [101]:
# LIBRARY_PATH = '/home/deleidos/Notebooks/Newspaper/SemanticEstimation'
LIBRARY_PATH = "/home/deleidos/Notebooks/lib"

import sys
import os

module_path = os.path.abspath(os.path.join(LIBRARY_PATH))
if module_path not in sys.path:
    sys.path.append(module_path)
print (f"sys.path : {sys.path}") 

sys.path : ['/home/deleidos/Notebooks/Newspaper/SemanticEstimation', '/opt/conda/lib/python38.zip', '/opt/conda/lib/python3.8', '/opt/conda/lib/python3.8/lib-dynload', '', '/opt/conda/lib/python3.8/site-packages', '/opt/conda/lib/python3.8/site-packages/IPython/extensions', '/home/deleidos/.ipython', '/home/deleidos/Notebooks/lib']


In [102]:
from APIutils import *
import newspaper
from newspaper import Article 
from keybert import KeyBERT

import json 
import boto3 
import logging 
import uuid 
import time
from datetime import date, datetime 
import datetime
from pathlib import Path 
import os 
import glob
import nltk
# from bertopic import BERTopic
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

from transformers import pipeline

import pandas as pd
import contractions

# nltk.download('punkt')
# setup widget import
from ipywidgets import widgets,Layout
from IPython.display import display

from utils import *
import random
from timeit import default_timer as timer
import time
import datetime


#LOCALS
RESOURCE = 'dynamodb'
TABLE = "Articles-tfoo6wfq2zbxfdwl3z47nsgtrq-dev"
REGION = "us-east-1"

CSVPath = "/home/deleidos/Notebooks/CSVData"
JSONPath = "/home/deleidos/Notebooks/JSONData"
MODELPATH= "/home/deleidos/Notebooks/Models"
# TRANSFORMERMODEL = 'bert-base-nli-mean-tokens'
from dynamoDButils import *
from APIutils import *
from seedterms import *


def complete(state="complete"):
    print(f"\nCell {state} @ {(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))}")
complete(state='imports done')


Cell imports done @ 2023-03-13 18:05:20


### Define the 3 different scoring methods

Create dictionary to hold scores and topics to eventually be displayed in a df.

- Old scoring method is the original method used to select articles for the newsfeed site. 
- The restrictive scoring (or modified BoW) approach was developed during researching/evaluating methods for the Semantic Estimation paper.
- The weighted scoring method provides a potential median to these aforementioned techniques.

In [103]:
# Create empty dict to add the scores to 
test_dict = {
    "summary":[], "oldScore":[], "modBoW":[], "weightScore":[], "topic1":[], "topic2":[]
}

# # Create empty dict to hold classifier results
# classify = {
#     "summary":[], "labels":[], "scores":[]
# }

In [104]:
# Original method for scoring 
def getOldScore(classify):
    THRESHOLD = 0.8
    
    score = round (
        (   round (classify['scores'][classify['labels'].index('technology')], 2 )
               + round (classify['scores'][classify['labels'].index('innovation')], 2 ) 
               + round (classify['scores'][classify['labels'].index('politics')], 2 )
               + round (classify['scores'][classify['labels'].index('electronics')], 2 )
               + round (classify['scores'][classify['labels'].index('federal')], 2 )
               + round (classify['scores'][classify['labels'].index('public')], 2 )
               + round (classify['scores'][classify['labels'].index('government')], 2 )
               + round (classify['scores'][classify['labels'].index('data')], 2 )
               + round (classify['scores'][classify['labels'].index('ai')], 2 )
               + round (classify['scores'][classify['labels'].index('application')], 2 )
               + round (classify['scores'][classify['labels'].index('security')], 2 )
               + round (classify['scores'][classify['labels'].index('cyber security')], 2 )
               + round (classify['scores'][classify['labels'].index('business innovation')], 2 )
            )
        ,2)
    
    return score
    

In [105]:
# Method stemming from modified BoW approach
def restrictiveScore(df):
    THRESHOLD = 0.4

    cumulative = 0.0
    topic1 = df.iloc[0] ['label']
    topic2 = df.iloc[1] ['label']
    score1 = round(df.iloc[0] ['score'],2)
    score2 = round(df.iloc[1] ['score'],2)
    cumulative = score1 + score2
    
    return (topic1, topic2, cumulative)

def format_results(classify):
    labels = classify['labels']
    scores = classify['scores']
    scores_csv = []
    for l in enumerate(classify['labels']):
        # note labels are in the form (0, 'public'), so can just use the oth term in l as index to scores
        scores_csv.append(
            {
                "label" : l[1],
                "score" : scores[l[0]]  
            }
        )
    return pd.DataFrame(scores_csv)

In [106]:
def weightedScore(classify):
    # Weighted topic ranking scoring system 
    CUTOFF = 0.45

    # multiplier for each topic score is indicative of the level of importance of the category
    weight_score = round ((
        (classify['scores'][classify['labels'].index('technology')] * 1.0) +
        (classify['scores'][classify['labels'].index('innovation')] * 1.0) +
        (classify['scores'][classify['labels'].index('ai')] * 0.8) +
        (classify['scores'][classify['labels'].index('business innovation')] * 0.8) +
        (classify['scores'][classify['labels'].index('science')] * 0.8) +
        (classify['scores'][classify['labels'].index('federal')] * 0.6) +
        (classify['scores'][classify['labels'].index('data')] * 0.6) +
        (classify['scores'][classify['labels'].index('cyber security')] * 0.6) +
        (classify['scores'][classify['labels'].index('government')] * 0.6) +
        (classify['scores'][classify['labels'].index('security')] * 0.5) +
        (classify['scores'][classify['labels'].index('electronics')] * 0.5) +
        (classify['scores'][classify['labels'].index('networks')] * 0.5) +
        (classify['scores'][classify['labels'].index('public')] * 0.4) +

        # labels indicative of off-topic articles given a negative multiplier
        (classify['scores'][classify['labels'].index('politics')] * -0.4) +
        (classify['scores'][classify['labels'].index('social')] * -0.6) +
        (classify['scores'][classify['labels'].index('games')] * -0.8) +
        (classify['scores'][classify['labels'].index('entertainment')] * -0.8) +
        (classify['scores'][classify['labels'].index('sports')] * -0.8) +
        (classify['scores'][classify['labels'].index('advertisement')] * -1.0) 
        ), 2)

    return (weight_score)


### Run corpus articles through the scoring methods

Load corpus from AWS DB, then run summaries through each score method to compare results.

In [107]:
%%time

RESOURCE = 'dynamodb'
TABLE = "Articles-tfoo6wfq2zbxfdwl3z47nsgtrq-dev"
REGION = "us-east-1"

os.environ['AWS_DEFAULT_REGION']= region
print (f"env : {os.getenv('AWS_DEFAULT_REGION')}")
print('\n\nconnecting ...')
exists = table_exists(RESOURCE, REGION, TABLE)
complete(state= "DB read")

env : us-east-1


connecting ...


KeyboardInterrupt: 

In [8]:
resp = extendedScanArticlesWithKey(RESOURCE,TABLE, columns="title,summary", keyword="url")

page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 5
page length : 4
extendedScanArticles: Length: 4626 


In [9]:
# Serializing json
indx = 0
jsndict = {}
for item in resp:
    jsndict[str(indx)]= {
        "url" : item['url']['S'],
        "title" : item['title']['S'],
        "summary": item['summary']['S']
    }
    indx += 1
    
json_object = json.dumps(jsndict, indent=4)
jsonfile = join(JSONPath, "semanticSimArticles.json")


with open(jsonfile, "w") as outfile:
     outfile.write(json_object)

In [10]:
jsonfiles = [f for f in listdir(JSONPath) if isfile(join(JSONPath, f))]
jsonfiles.sort()
jsonfiles = [x for x in jsonfiles if (os.path.splitext(x)[1][1:]).lower() == "json"]
jsnWD = widgets.Select(
    options=jsonfiles,
    value=jsonfiles[0],
    description='Choose ID:',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
        layout=Layout(width='50%', height='256px')
)
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
#         print ("changed to %s" % change['new'])
        selection = change['new']

jsnWD.observe(on_change)
display(jsnWD)

Select(description='Choose ID:', layout=Layout(height='256px', width='50%'), options=('2000CWE.json', 'NOTsema…

In [11]:
jsnf = jsnWD.value
print (f"{jsnf=} ")

jsnf='semanticSimArticles.json' 


In [108]:
jsnfile = osp.join (JSONPath,jsnf)
df = pd.read_json(jsnfile)
xposedf = df.transpose() # pivot the matrix
xposedf

,url,title,summary,corpusAge,clicks
0,https://www.wired.com/story/white-house-nation...,The High-Stakes Blame Game in the White House ...,In the endless fight to improve cybersecurity ...,3,0
1,https://www.cnbc.com/2023/02/23/baidu-says-its...,Baidu says its alternative to ChatGPT is comin...,In this article BIDU Follow your favorite stoc...,14,0
2,https://arstechnica.com/gadgets/2023/03/revamp...,"New Windows 11 preview improves volume mixer, ...",Microsoft released a batch of significant upda...,5,0
3,https://meritalk.com/articles/house-panel-push...,House Panel Pushes for Data Privacy Legislation,While the House Energy and Commerce Committee ...,3,0
4,https://federalnewsnetwork.com/cybersecurity/2...,"For secure and reliable applications, separate...",Few operations are both as common and as risky...,40,0
...,...,...,...,...,...
3529,https://blog.executivebiz.com/2023/02/lockheed...,"Lockheed, Juniper Demo Routing Technology to O...",Lockheed Martin has partnered with Juniper Net...,10,0
3530,https://www.techradar.com/deals/buy-the-meta-q...,Buy the Meta Quest 2 and get two free games wi...,I've been a little obsessed with playing golf ...,30,0
3531,https://www.govconwire.com/2023/02/stephen-hag...,Army Veteran Stephen Hager Becomes 1st CISO of...,"Stephen Hager, a 34-year U.S. Army veteran, ha...",8,0
3532,http://www.foxbusiness.com/media/princeton-stu...,Princeton student develops app to eliminate Ch...,ChatGPT's meteoric rise has certainly opened t...,25,0


In [109]:
xSentences = preProcessing(xposedf)


Cell models @ 2023-03-13 18:07:51


In [110]:
xSentences[0]

'endless fight improve cybersecurity encourage investment digital defense expert controversial suggestion say way make company take seriously create real economic incentive making legally liable taken adequate step secure product infrastructure last thing anyone want liability idea never exploded popularity national cybersecurity strategy white house week giving concept prominent boost long awaited document proposes stronger cybersecurity protection regulation critical infrastructure expanded program disrupt cybercriminal activity focus global cooperation many priority widely accepted build national strategy put past administration biden strategy expands significantly question liability must begin shift liability onto entity fail take reasonable precaution secure software recognizing even advanced software security program cannot prevent vulnerability say company make software must freedom innovate must also held liable fail live duty care owe consumer business critical infrastructure 

In [ ]:
# Create a zero shot classifier with specific category labels
# from transformers import pipeline - testing two different models to assess the run-time
classifier = pipeline("zero-shot-classification"
#                     ,model="facebook/bart-large-mnli" 
                    ,model="valhalla/distilbart-mnli-12-1" 
                    ,device=0                   
                     ) # to utilize GPU specify device= ...

complete(state='classifier initialised')

In [97]:
# Long list of labels (22 topics)
# candidate_labels = [ 'technology', 'innovation','politics', 'electronics'
#                     ,'entertainment', 'economics', 'advertisement', 'social'
#                     ,'games', 'sports', 'health', 'science', 'business innovation'
#                     ,'federal', 'public', 'government', 'data', 'ai'
#                     , 'application', 'security', 'cyber security', 'networks' ]

# short list of labels used in modBoW experiment (13 topics)
# candidate_labels = ['technology', 'innovation', 'politics', 'electronics','entertainment', 'economics'
#                     , 'advertisement', 'social', 'games'
#                     , 'sports', 'health', 'science'
#                     , 'business innovation']

# proposed medium list (16 topics)
candidate_labels = [ 'technology', 'innovation', 'politics'
                    ,'entertainment', 'economics', 'advertisement', 'social'
                    ,'games', 'sports', 'health', 'science', 'business innovation'
                    ,'federal', 'data', 'ai' ,'cyber security']

First run each summary through the classifier, then score the article with each method outlined above. 

Add the results to a dictionary for eventual display in a DF.

In [98]:
def loadCandidateArticlesTest(summaries):
    progress_bar = widgets.IntProgress(min=0, max=len(summaries), description="Extracting")
    display(progress_bar)
    classify = []
    
    for summary in summaries:
        article = classifier(summary, candidate_labels ,device=0)
        classify.append(article)
#         classify['summary'].append(summary)
#         classify['labels'].append(article['labels'])
#         classify['scores'].append(article['scores'])
        progress_bar.value += 1
    
    progress_bar.bar_style = "success"
    progress_bar.close()
    return classify

In [99]:
%%time
classify_test = loadCandidateArticlesTest(xSentences[:100])

IntProgress(value=0, description='Extracting')

CPU times: user 4min 25s, sys: 1min 41s, total: 6min 7s
Wall time: 6min 3s


In [100]:
classify_test[0:5]

[{'sequence': 'studio ghibli founder director hayao miyazaki responsible beloved anime classic like princess mononoke kiki delivery service made headline around world reaction animation program would never wish incorporate technology work miyazaki told software engineer came show creation strongly feel insult life half decade later artificial intelligence potential role could play anime production spotlight week netflix shared dog boy animated short streaming giant described experimental effort address anime industry ongoing labor shortage used image generation technology background image three minute video cut said netflix japan project twitter according machine translation short touching immediately controversial motherboard point many twitter user accused netflix using avoid paying human artist others took issue netflix wit studio company produced short credited worked dog boy see end video human animator involved creation short background also revised work however background design

In [19]:
%%time
classify2 = loadCandidateArticles(xSentences)

IntProgress(value=0, description='Extracting', max=4626)

CPU times: user 4h 33min 10s, sys: 2h 22min 28s, total: 6h 55min 39s
Wall time: 6h 52min 24s


In [73]:
classify = classify2
classify[0:5]

[{'sequence': 'studio ghibli founder director hayao miyazaki responsible beloved anime classic like princess mononoke kiki delivery service made headline around world reaction animation program would never wish incorporate technology work miyazaki told software engineer came show creation strongly feel insult life half decade later artificial intelligence potential role could play anime production spotlight week netflix shared dog boy animated short streaming giant described experimental effort address anime industry ongoing labor shortage used image generation technology background image three minute video cut said netflix japan project twitter according machine translation short touching immediately controversial motherboard point many twitter user accused netflix using avoid paying human artist others took issue netflix wit studio company produced short credited worked dog boy see end video human animator involved creation short background also revised work however background design

In [23]:
for indx in range(len(classify)): 
    score = getOldScore(classify[indx])
    
    df = pd.DataFrame()
    df = format_results(classify[indx])
    rtn = restrictiveScore(df)
    
    weight_score = weightedScore(classify[indx])
    
    # Add the re-evalaution scores and summary to dictionary
    test_dict['summary'].append(classify[indx]['sequence'])
    test_dict['oldScore'].append(score)
    test_dict['modBoW'].append(rtn[2])
    test_dict['weightScore'].append(weight_score)
    test_dict['topic1'].append(rtn[0])
    test_dict['topic2'].append(rtn[1])
       

In [24]:
# convert dictionary to pandas df
scoresdf = pd.DataFrame(test_dict)
scoresdf.head(15)

,summary,oldScore,modBoW,weightScore,topic1,topic2
0,studio ghibli founder director hayao miyazaki ...,0.79,0.47,0.45,public,technology
1,article bidu follow favorite stock create free...,0.89,0.29,0.49,business innovation,public
2,chatgpt passed exam wharton business school pr...,0.86,0.52,0.58,technology,public
3,waiting apple unveil long rumored headset take...,0.66,0.22,0.28,ai,networks
4,microsoft released batch significant update wi...,0.80,0.53,0.34,public,technology
5,ibm think tank emphasized need quantum safe st...,0.91,0.37,0.53,cyber security,security
6,operation common risky deploying new applicati...,0.91,0.34,0.50,security,technology
7,winter covid surge fading fast likely thanks w...,0.64,0.35,0.26,public,data
8,prowess openai chatgpt continues expand beyond...,0.86,0.45,0.52,technology,application
9,chatgpt excited investor imagination according...,0.75,0.32,0.53,innovation,technology


In [25]:
# add url and title to df
url = xposedf['url']
title = xposedf['title']
scoresdf.insert(0, 'url', url)
scoresdf.insert(1, 'title', title)

In [35]:
scoresdf

,url,title,summary,oldScore,modBoW,weightScore,topic1,topic2
0,https://www.engadget.com/netflixs-dog-and-boy-...,Netflix's 'Dog and Boy' anime causes outrage f...,studio ghibli founder director hayao miyazaki ...,0.79,0.47,0.45,public,technology
1,https://www.cnbc.com/2023/02/23/baidu-says-its...,Baidu says its alternative to ChatGPT is comin...,article bidu follow favorite stock create free...,0.89,0.29,0.49,business innovation,public
2,https://businessinsider.com/wharton-professor-...,Wharton professor who gave ChatGPT an MBA-leve...,chatgpt passed exam wharton business school pr...,0.86,0.52,0.58,technology,public
3,https://www.macworld.com/article/1484137/apple...,Apple’s next battle in its ‘silent war’ agains...,waiting apple unveil long rumored headset take...,0.66,0.22,0.28,ai,networks
4,https://arstechnica.com/gadgets/2023/03/revamp...,"New Windows 11 preview improves volume mixer, ...",microsoft released batch significant update wi...,0.80,0.53,0.34,public,technology
...,...,...,...,...,...,...,...,...
4621,https://blog.executivebiz.com/2023/02/lockheed...,"Lockheed, Juniper Demo Routing Technology to O...",lockheed martin partnered juniper network demo...,0.90,0.58,0.74,technology,public
4622,https://www.techradar.com/deals/buy-the-meta-q...,Buy the Meta Quest 2 and get two free games wi...,little obsessed playing golf switch sport late...,0.41,0.45,-0.14,sports,technology
4623,https://www.govconwire.com/2023/02/stephen-hag...,Army Veteran Stephen Hager Becomes 1st CISO of...,stephen hager year army veteran appointed newl...,0.90,0.52,0.45,security,cyber security
4624,http://www.foxbusiness.com/media/princeton-stu...,Princeton student develops app to eliminate Ch...,chatgpt meteoric rise certainly opened door en...,0.89,0.52,0.61,public,technology


In [42]:
# Create a new df that only displays articles that are greater than the score cut-off of 0.45
rescoredf = scoresdf.copy()
rescoredf.drop(['oldScore', 'modBoW'], axis=1)

newdf = rescoredf.loc[(rescoredf['weightScore'] >= 0.45) , 
                      ['url', 'title', 'summary', 'weightScore', 'topic1', 'topic2']
    ]

In [43]:
newdf

,url,title,summary,weightScore,topic1,topic2
0,https://www.engadget.com/netflixs-dog-and-boy-...,Netflix's 'Dog and Boy' anime causes outrage f...,studio ghibli founder director hayao miyazaki ...,0.45,public,technology
1,https://www.cnbc.com/2023/02/23/baidu-says-its...,Baidu says its alternative to ChatGPT is comin...,article bidu follow favorite stock create free...,0.49,business innovation,public
2,https://businessinsider.com/wharton-professor-...,Wharton professor who gave ChatGPT an MBA-leve...,chatgpt passed exam wharton business school pr...,0.58,technology,public
5,https://blog.executivebiz.com/2023/01/ibm-thin...,IBM Think Tank Highlights Quantum-Safe Strateg...,ibm think tank emphasized need quantum safe st...,0.53,cyber security,security
6,https://federalnewsnetwork.com/cybersecurity/2...,"For secure and reliable applications, separate...",operation common risky deploying new applicati...,0.50,security,technology
...,...,...,...,...,...,...
4615,https://www.govconwire.com/2023/03/altamira-bu...,Altamira Adds SIGINT Services Through VaST Acq...,altamira technology acquired signal intelligen...,0.76,technology,security
4619,http://newsroom.morningstar.com/newsroom/news-...,"How are Data, Direct, and Advisor Workstation ...",primarily deliver data client via data feed ft...,0.47,application,technology
4621,https://blog.executivebiz.com/2023/02/lockheed...,"Lockheed, Juniper Demo Routing Technology to O...",lockheed martin partnered juniper network demo...,0.74,technology,public
4623,https://www.govconwire.com/2023/02/stephen-hag...,Army Veteran Stephen Hager Becomes 1st CISO of...,stephen hager year army veteran appointed newl...,0.45,security,cyber security


In [40]:
# Calculate percentage of corpus that passed new scoring method
percent_passed = (len(newdf)/len(rescoredf))*100

print (f"Percent Passed : {percent_passed}%")

Percent Passed : 34.522265456117594%


### Write results and data to JSON
- articleScoringExperiment.json contains the results from all 3 scoring methods for the whole corpus
- articleScoringExperimentData.json contains raw classifier data (as a back-up copy since it took so long to run)

In [27]:
jsonfile = join(JSONPath, "articleReScoringExperiment.json")
print(jsonfile)

scoresdf.to_json(jsonfile)
complete(state="updates written: ")

/home/deleidos/Notebooks/JSONData/articleReScoringExperiment.json

Cell updates written:  @ 2023-03-04 22:23:53


In [30]:
jclassify = json.dumps(classify, indent=4)

In [32]:
jsonfile = join(JSONPath, "articleReScoringExperimentData.json")
print(jsonfile)
with open(jsonfile, "w") as outfile:
     outfile.write(jclassify)

complete(state="updates written: ")

/home/deleidos/Notebooks/JSONData/articleReScoringExperimentData.json

Cell updates written:  @ 2023-03-04 22:30:37
